In [ ]:
1+1

In [1]:
using FunctionalCollections
using Iterators
using Pipe
using Compat

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")
using DataStructures
using DataStructuresExtended
using JLD
using DataFrames

INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/DataStructures.ji for module DataStructures.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/Docile.ji for module Docile.
INFO: Recompiling stale cache file /home/ubuntu/.julia/lib/v0.5/DataFrames.ji for module DataFrames.


In [ ]:
OUTPUT_DIR = "../../documents_prepared/Journal\ Papers/SumResynthesis/data/"

In [ ]:
`git add ../../documents_prepared/Journal\ Papers/SumResynthesis/data/ordering_scores.csv` |> run

In [ ]:
`git push` |> run

In [ ]:
function jaccard_index(aa::DataStructures.Accumulator,bb::DataStructures.Accumulator)
    sum(intersect(aa,bb))/sum(union(aa,bb))
end

function jaccard_index(aa,bb)
    length(intersect(aa,bb))/length(union(aa,bb))
end

In [ ]:


brown_glove300_raw = load("results/bags/brown_g50D_res.jld", "res")
brown_glove200_raw = load("results/bags/brown_glove200_res.jld", "res")
brown_glove100_raw = load("results/bags/brown_glove100_res.jld", "res")
brown_glove50_raw = load("results/bags/brown_glove50_res.jld", "res")

In [ ]:
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove300_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove200_raw])
@assert all([rset[1] for rset in brown_glove50_raw] .== [rset[1] for rset in brown_glove100_raw])

In [ ]:
bag_res = DataFrame()
bag_res[:ground] = [counter(rset[1]) for rset in brown_glove50_raw]
bag_res[:ground_len] = Int[sum(ss) for ss in bag_res[:ground]]
exp_names = ASCIIString[]
for raw_name in ["brown_glove50_raw","brown_glove100_raw", "brown_glove200_raw", "brown_glove300_raw"]
    colname = split(raw_name,"_")[2] 
    push!(exp_names, colname)
    raw = eval(raw_name|>Symbol)
    bag_res[colname*"_actual"|> Symbol] = [counter(rset[2]) for rset in raw]
    #bag_res[colname*"_distance"|> Symbol] = Float64[-1*rset[3] for rset in raw]
end

bag_res

In [ ]:
for exp_name in exp_names
    actual = bag_res[exp_name*"_actual" |> symbol]
    bag_res[exp_name*"_perfect" |> symbol] = actual.==bag_res[:ground]
    bag_res[exp_name*"_jaccard" |> symbol] = map(jaccard_index, actual, bag_res[:ground])
end

In [ ]:

jaccard_cols = Symbol[exp_name*"_jaccard" |> symbol for exp_name in exp_names]
perfect_cols = Symbol[exp_name*"_perfect" |> symbol for exp_name in exp_names]
len_scores = aggregate(bag_res[[:ground_len, jaccard_cols...,perfect_cols...]], :ground_len, mean)
sort!(len_scores)
writetable(OUTPUT_DIR*"selection_len_scores.csv", len_scores)
len_scores

In [ ]:
cum_len_scores = DataFrame()
cum_len_scores[:max_ground_len] = bag_res[:ground_len] |> unique |> sort!
for jaccard_col in jaccard_cols
    cum_len_scores[jaccard_col] = Float64[ begin
            shortish_sents = bag_res[:ground_len] .<= max_len;
            bag_res[shortish_sents, jaccard_col] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end
for perfect_col in perfect_cols
    cum_len_scores[perfect_col] = Float64[ begin
            shortish_sents = bag_res[:ground_len] .<= max_len;
            bag_res[shortish_sents, perfect_col] |> mean
         end for max_len in cum_len_scores[:max_ground_len]]
end
writetable(OUTPUT_DIR*"selection_cum_len_scores.csv", cum_len_scores)
cum_len_scores 


In [ ]:
function longform(df::DataFrame, index::Symbol, cols::Vector{Symbol}, new_name::Symbol)
    function subdf(col::Symbol)
        df_sub = df[[index, col]]
        rename!(df_sub, col, new_name)
        df_sub[:label] = col
        df_sub
    end
    vcat([subdf(col) for col in cols]...)
end




In [ ]:
using Gadfly

In [ ]:
p=Gadfly.plot(longform(cum_len_scores, :max_ground_len, jaccard_cols, :mean_jaccard_index),
    x="max_ground_len", y="mean_jaccard_index",
    color="label",
    Geom.point
)
#draw(PNG("brown_cumulative_length_score.png", 24cm, 6cm), p)

In [ ]:

p=Gadfly.plot(longform(len_scores, :ground_len, [string(col)*"_mean"|> Symbol for col in jaccard_cols], :mean_jaccard_index),
    x="ground_len", y="mean_jaccard_index",
    color="label",
    Geom.point
)
#draw(PNG("brown_length_score.png", 24cm, 6cm), p)


In [ ]:
bag_res[:ground_len] |> describe 
p=Gadfly.plot(bag_res, x="ground_len",  Geom.histogram)
#draw(PNG("brown_length_dist.png", 24cm, 6cm), p)

In [ ]:
##############################################
# ORDERED
########################################

In [2]:
using PyCall
@pyimport nltk
@pyimport nltk.align.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(candidate,Any[reference], weights)
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


bleu_score (generic function with 1 method)

In [3]:
bleu_score(["the","comedy","of","manners","of","this","first","a","basis","is","performed","in","1892","."],
           ["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."]
)

0.30576902884505114

In [55]:
#folds50 = load("results/ordered/brown_glove50_ordered_vA1000.jld")
#folds100 = load("results/ordered/brown_glove100_ordered_vA1000.jld")
#folds200 = load("results/ordered/brown_glove200_ordered_vA1000.jld")
folds300_loglikelyhood = load("results/ordered/brown_glove300_ordered_vA1000_loglikelyhood.jld")
folds300 = load("results/ordered/brown_glove300_ordered_vA1000.jld")
#folds_oracle = load("results/ordered/brown_oracle_ordered_vA1000.jld")
#folds_oracle_loglikelyhood = load("results/ordered/brown_oracle_ordered_vA1000_Loglikelyhood.jld")
all_ordered_exp_folds = Any[folds_oracle, folds_oracle_loglikelyhood,folds300,folds300_loglikelyhood]#[folds50,folds100,folds200,folds300,folds_oracle]

4-element Array{Any,1}:
 Dict{ByteString,Any}("fold_6"=>Any[(ASCIIString["the","songs","sandburg","sang","often","reminded","listeners","of","songs","of","a","kindred","character","they","knew","entirely","or","in","fragments","."],ASCIIString["entirely","songs","the","fragments","reminded","often","kindred","knew","sandburg","a","listeners","of",".","or","character","sang","they","in","of","songs"],NaN),(ASCIIString["it","will","help","to","determine","the","attitude","of","the","new","members","toward","the","church","."],Any["determine","the","new","members","of","the","church","will","help","to","the","attitude","toward","it","."],3.9715980931448507e-26),(ASCIIString["(","check","on","the","payne","luggage","."],ASCIIString["check","the","(","on",".","payne","luggage"],5.0),(ASCIIString["when","it","was","over",",","eight","of","his","company","were","still","alive","and","all","eight","were","wounded","."],Any["eight","was","still","alive",",","and","when","it","over","all","were"

4200-element Array{Any,1}:
 (ASCIIString["."],Any["."],0.001203892923635234)                                                                                                                                                                                                                                                                                                                                        
 (ASCIIString["but","he","warmed","up","after","a","while","."],ASCIIString["a","while","he","up","but","after",".","warmed"],7.0)                                                                                                                                                                                                                                                       
 (ASCIIString[")","what","it","does",":"],Any[")","what","it","does",":"],1.72709590345844e-9)                                                                                                                           

In [84]:
fold_id = "fold_3"
fo = folds_oracle[fold_id]
f3 = folds300[fold_id]
function exact_match_ele(rcp)
    (reference, candidate, prob)  = rcp
    0<prob<1 && reference==candidate
end

exact_matched_f3 = map(exact_match_ele, f3)
exact_matched_fo = map(exact_match_ele, fo)

oddballs = map(exact_matched_f3, exact_matched_fo) do f3_e, fo_e
    f3_e && !fo_e 
end
sum(oddballs)

1

In [86]:
find(oddballs)

1-element Array{Int64,1}:
 459

In [87]:
f3[459]

(ASCIIString["he","found","it",",","if","anything",",","all","too","familiar","."],Any["he","found","it",",","if","anything",",","all","too","familiar","."],7.831654154801354e-20)

In [88]:
fo[459]

(ASCIIString["he","found","it",",","if","anything",",","all","too","familiar","."],Any["if","anything",",","he","found","it",",","all","too","familiar","."],3.184459479527181e-20)

In [89]:
folds_oracle_loglikelyhood[fold_id][459]

(ASCIIString["he","found","it",",","if","anything",",","all","too","familiar","."],Any["if","anything",",","he","found","it",",","all","too","familiar","."],8.656258336662922e-20)

In [91]:
folds300_loglikelyhood[fold_id][459]

(ASCIIString["he","found","it",",","if","anything",",","all","too","familiar","."],Any["he","found","it",",","if","anything",",","all","too","familiar","."],2.1288643175772356e-19)

In [56]:
function get_bleu_score(folds)
    bleu_scores = map(keys(folds), values(folds)) do fold_name,res
        fold_bleu_scores = Float64[]
        for (reference, candidate, prob) in res
            push!(fold_bleu_scores, bleu_score(candidate, reference))
        end
        fold_bleu_scores
    end
end

get_bleu_score (generic function with 1 method)

In [72]:
attempted_only{T}(fold::Vector{T}) = filter(eap->!isnan(eap[3]), fold)
nongiveup_only{T}(fold::Vector{T}) = filter(eap->0<eap[3]<1, fold)
partial_only{T}(fold::Vector{T}) = filter(eap->eap[3]>1, fold)

attempted_only{T}(folds::Vector{Vector{T}}) = map(attempted_only, folds)
nongiveup_only{T}(folds::Vector{Vector{T}}) = map(nongiveup_only, folds)
partial_only{T}(folds::Vector{Vector{T}}) = map(partial_only, folds)

partial_only (generic function with 2 methods)

In [58]:
bleu_scores = map(all_ordered_exp_folds) do folds
    bleu_scores = get_bleu_score(folds);
    map(mean, bleu_scores) |> mean
end

4-element Array{Any,1}:
 0.102546
 0.102532
 0.101646
 0.101606

In [59]:
bleu_scores_no_giveup = map(all_ordered_exp_folds) do folds
    mean([mean([bleu_score(eap[1], eap[2]) for eap in nongiveup_only(attempted_only(fold))]) for fold in values(folds)])
end

4-element Array{Any,1}:
 0.504141
 0.504102
 0.50403 
 0.504061

In [60]:
giveups = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[!(0<eap[3]<1) for eap in attempted_only(fold)]) for fold in values(folds)])
end

4-element Array{Any,1}:
 0.715619
 0.715577
 0.718476
 0.718476

In [67]:
exact_matchs = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[1]==eap[2] for eap in attempted_only(fold)]) for fold in values(folds)])
end

4-element Array{Any,1}:
 0.129699
 0.129699
 0.129782
 0.129782

In [74]:
par_exact_matchs = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[3]>1 for eap in attempted_only(fold)]) for fold in values(folds)])
end

4-element Array{Any,1}:
 0.680739
 0.680697
 0.683468
 0.683468

In [62]:
exact_matchs_no_giveup = map(all_ordered_exp_folds) do folds
    mean([mean(Bool[eap[1]==eap[2] for eap in nongiveup_only(attempted_only(fold))]) for fold in values(folds)])
end

4-element Array{Any,1}:
 0.39119 
 0.391133
 0.392054
 0.392056

In [63]:
ordering_res = DataFrame(Model=["Oracle", "Oracle_LL"])#["50d","100d","200d","300d", "Oracle"])
ordering_res[symbol("BLEU Score")]=bleu_scores
ordering_res[symbol("Exact Ordered Matches")]=exact_matchs
ordering_res[symbol("Giveups")]=giveups
ordering_res[symbol("BLEU Score Excluding Giveups")]=bleu_scores_no_giveup
ordering_res[symbol("Exact Ordered Matches Excluding Giveups")]=exact_matchs_no_giveup
#writetable(OUTPUT_DIR*"ordering_scores.csv", ordering_res)
ordering_res

LoadError: LoadError: New columns must have the same length as old columns
while loading In[63], in expression starting on line 2

In [ ]:
########

In [ ]:
fold_indexes=load("brown_glove_folds.jld","fold_indexes");

In [ ]:
function fold(data, fold_indexes)
    ret = Dict()
    for (fold_number,fold_iis) in enumerate(fold_indexes)
        ret["fold_$(fold_number)"] = data[fold_iis]
    end
    ret
end

In [ ]:
jaccard_folds_300 = fold(bag_res[:glove300_jaccard], fold_indexes)

In [ ]:
[jaccard_folds_300["fold_6"] bleu_scores300[1]]

In [ ]:
get_bleu_score(folds300,  count_give_ups=true, count_nonattepts=true);